### TCP-2.1

In [ ]:
import socket
import sys

    
msg = "Hello, I'm the server"

server_address = ("localhost", 6780)
sock = socket.socket(family=socket.AF_INET, type=socket.SOCK_STREAM)
sock.bind(server_address)
sock.listen(1)
connection, client_address = sock.accept()
while True:

    command = connection.recv(1024).decode().split()

    print("RX", command)
    
    if command[0] == "bye":
        connection.sendall("Server shutting down".encode())
        
        connection.close()
        break
        
    elif command[0]=="get":
        file = command[1]
        try:
            with open(file) as f:
                contents = f.read()
                print("Contents: ", contents)
                msg = contents
                
        except:
            msg = "Error File not found."
    else:
        msg = "Invalid command"
    connection.sendall(msg.encode())



### TCP-2.2

In [5]:
import socket
import sys
import time

server_address = ("localhost", 6780)
sock = socket.socket(family=socket.AF_INET, type=socket.SOCK_STREAM)
sock.bind(server_address)
sock.listen(1)
connection, client_address = sock.accept()

lock_dict = {}
while True:
    time.sleep(1)
    command = connection.recv(1024).decode().split()
    print("RX", command)
    
    if command[0] == "bye":
        connection.sendall("Server shutting down".encode())
        
        connection.close()
        break
        
    elif command[0]=="get":
        file = command[1]
        
        try:
            file_lock = str(lock_dict[file])
        except:
            file_lock = "" # File no bloqueado por nadie
        
        try:
            if file_lock == str(client_address) or file_lock == "" : # Comprueba que no esté bloqueado por otro cliente
                    
                with open(file) as f:
                    contents = f.read()
                    print("Contents: ", contents)
                    msg = contents
            else:
                msg = "Error File is locked by another client"
        except Exception as e:
            print(e)
            msg = "Error File not found."     
       
    elif command[0]=="lock":
        file = command[1]
        
        try:
            f = open(file, "r")
            f.close()
            lock_dict[file] = client_address
            msg = "File "+ file + " locked succesfully by client "+ str(client_address)
        except Exception as e:
            print(e)
            msg = "File not found"
            
    elif command[0]=="put":
        file = command[1]
        file_contents = " ".join(command[2:])
        print("UPDATED FILE CONTENTS: ", file_contents)

        try:
            file_lock = str(lock_dict[file])
        except Exception as e:
            file_lock = "" # File no bloqueado por nadie
        try:
            
            if file_lock == str(client_address): # Comprueba que el archivo está bloqueado por el cliente
                    
                f = open(file, "w+")
                f.write(file_contents)
                f.close()
                msg = "File updated successfully"
            else:
                msg = "File needs to be locked first in order to be updated"
        except Exception as e:
            msg = "File not found." 
         
    elif command[0]=="unlock":
        file = command[1]

        try:
            f = open(file, "r")
            f.close()
            lock_dict[file] = ""
            msg = "File "+ file + " unlocked succesfully by client "+ str(client_address)
        except Exception as e:
            print(e)
            msg = "File not found"
        
    else:
        msg = "Invalid command"
    connection.sendall(msg.encode())



RX ['put', 'file.txt', 'File', 'file.txt', 'retrieved']
FILE CONTENTS File file.txt retrieved
RX ['get', 'file.txt']
Contents:  File file.txt retrieved
RX ['put', 'file.txt', 'File', 'edited', 'by', 'the', 'client']
FILE CONTENTS File edited by the client
RX ['lock', 'f']
[Errno 2] No such file or directory: 'f'
RX ['lock', 'file.txt']
RX ['put', 'file.txt', 'File', 'edited', 'by', 'the', 'client']
FILE CONTENTS File edited by the client
RX ['get', 'file.txt']
Contents:  File edited by the client
RX ['unlock', 'file.txt']
RX ['put', 'file.txt', 'File', 'edited', 'by', 'the', 'client']
FILE CONTENTS File edited by the client
RX ['bye']
